# H.02 | Introduction to SQL and OLAP

H.02 will serve as a simple introduction to SQL and OLAP. It will cover the basic concepts and provide a foundation for understanding how to work with databases and perform data analysis using SQL. We'll use Snowflake as our database and TPC-H as our dataset.

## Snowflake

Snowflake is a cloud-based data warehousing platform that provides a powerful and flexible environment for storing, processing, and analyzing large volumes of data. It is designed to handle complex data workloads and offers features such as scalability, high performance, and ease of use. Snowflake supports SQL as its primary query language, making it accessible to users familiar with SQL. We will be using Snowflake to demonstrate the concepts of SQL and OLAP.

Please ensure you have the .env file filled out in the root of this folder, it is required to connect to Snowflake.

## TPC-H 

TPC-H is a dataset that simulates a real-world business environment, and has relatively simple schema. It is widely used for testing and comparing the performance of different database systems. We're going to use TPC-H to demonstrate the concepts of SQL and OLAP. The dataset consists of several tables, each representing a different aspect of the business. The tables are related to each other through foreign keys, which allow us to join them together and perform complex queries.

<div style="align: center; justify-content: center; display: flex;">
    <img src="https://docs.snowflake.com/en/_images/sample-data-tpch-schema.png" alt="Snowflake Schema" width="400" height="400" style = "border-radius: 10px">
</div>

## Snowflake Connection

To connect to Snowflake, we will use the `snowflake-connector-python` library. This library provides a simple and efficient way to connect to Snowflake and execute SQL queries. We will also use the `pandas` library to load the data into a DataFrame for easy viewing.

**NOTE**: Snowflake will send you a MFA code when you run the below cell. If it doesn't, make sure you can login on the web interface.

In [1]:
import sys
sys.path.append('../')
from connection import connect_to_snowflake, as_dataframe
conn = connect_to_snowflake(database="SNOWFLAKE_SAMPLE_DATA", schema="TPCH_SF1")
cursor = conn.cursor()

/workspaces/SPRING25-DATA-INTENSIVE-SYSTEMS/venv/lib/python3.10/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


# Exercise 1 | Basic SQL Practice

This part of the homework will focus on using the Snowflake Connector for Python to connect to a Snowflake database and perform basic operations.

## Exercise 1.1

Retrieve the names and account balances of all customers whose balance is greater than 5000. Limit to 5 rows and order by acctbal (ascending).

You should output a table with **5 rows** that looks like:

| C_NAME | C_ACCTBAL |
| ------- | --------- |
| Customer#000095488 | 5000.09 |
| ... | ... |

In [2]:
QUERY_CHALLENGE_1 = """
SELECT c_name, c_acctbal FROM customer
WHERE c_acctbal > 5000
ORDER BY c_acctbal ASC
LIMIT 5;
"""
cursor.execute(QUERY_CHALLENGE_1)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

            C_NAME C_ACCTBAL
Customer#000095488   5000.09
Customer#000146711   5000.21
Customer#000024740   5000.23
Customer#000075576   5000.27
Customer#000103760   5000.29


## Exercise 1.2

List orderkey, orderdate, and totalprice with an order date in January 1995. Limit to 5 rows and order by totalprice (descending).

You should output a table with **5 rows** that looks like:

| O_ORDERKEY | O_ORDERDATE | O_TOTALPRICE | 
| ----------- | ----------- | ----------- |
| 2948355 | 1995-01-21 | 469472.71 |
| ... | ... | ... |

In [3]:
QUERY_CHALLENGE_2 = """
SELECT o_orderkey, o_orderdate, o_totalprice FROM orders
WHERE MONTH(o_orderdate) = 1 AND YEAR(o_orderdate) = 1995
ORDER BY o_totalprice DESC
LIMIT 5;
"""
cursor.execute(QUERY_CHALLENGE_2)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

 O_ORDERKEY O_ORDERDATE O_TOTALPRICE
    2948355  1995-01-21    469472.71
    5501606  1995-01-29    454827.49
    4806726  1995-01-17    446704.09
    4978885  1995-01-12    446578.70
    3037414  1995-01-25    443807.22


## Exercise 1.3

Find the total number of parts via unique part keys. Rename the count to "count".

You should output a table with **1 rows** that looks like:

| COUNT |
| ----------- |
| 200000 |

In [4]:
QUERY_CHALLENGE_3 = """
SELECT COUNT(p_partkey) AS count FROM part; 
"""
cursor.execute(QUERY_CHALLENGE_3)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

 COUNT
200000


## Exercise 1.4

List the top 5 supplier names and account balances with the highest account balances. Limit to 5 rows in descending order of account balances.

You should output a table with **5 rows** that looks like:

| S_NAME | S_ACCTBAL |
| ----------- | ----------- |
| Supplier#000009450 | 9999.72 |
| ... | ... |

In [5]:
QUERY_CHALLENGE_4 = """ 
SELECT s_name, s_acctbal FROM supplier
ORDER BY s_acctbal DESC
LIMIT 5;
"""
cursor.execute(QUERY_CHALLENGE_4)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

            S_NAME S_ACCTBAL
Supplier#000009450   9999.72
Supplier#000006343   9998.20
Supplier#000002522   9997.04
Supplier#000000892   9993.46
Supplier#000002543   9992.70


## Exercise 1.5

Calculate the average order price across all orders. Rename the column to `AVG_ORDER_PRICE`.

You should output a table that looks like:
| AVG_ORDER_PRICE |
| ----------- |
| ... |

In [6]:
QUERY_CHALLENGE_5 = """ 
SELECT AVG(o_totalprice) AS AVG_ORDER_PRICE FROM orders; 
"""
cursor.execute(QUERY_CHALLENGE_5)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

AVG_ORDER_PRICE
151219.53763164


## Excercise 1.6

Retrieve the p_name and p_partkey of all parts with a name that contains the word "red". Limit to 10 rows and sort by p_partkey (ascending).

You should output a table with **10 rows** that looks like:

| P_NAME | P_PARTKEY |
| ----------- | ----------- |
| honeydew red azure magenta brown   | 47 |
| ... | ... |

In [7]:
QUERY_CHALLENGE_6 = """ 
SELECT p_name, p_partkey FROM part
WHERE p_name LIKE '%red%'
ORDER BY p_partkey ASC
LIMIT 10;
"""
cursor.execute(QUERY_CHALLENGE_6)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

                          P_NAME  P_PARTKEY
honeydew red azure magenta brown         47
goldenrod black slate forest red        116
 tomato honeydew pale red yellow        117
        white red lace deep pale        187
    peach cornsilk navy rosy red        200
cyan aquamarine red plum frosted        219
seashell tomato red lemon saddle        233
 drab aquamarine red papaya pale        248
      navy light red royal olive        254
     linen red plum purple steel        263


## Exercise 1.7

Retrieve the names of customers who have orders with a total price greater than 100,000. Limit to 10 rows and order by c_name.

You should output a table that looks like:

| C_NAME |
| ----------- |
| Customer#000000001 |
| ... |

In [10]:
QUERY_CHALLENGE_7 = """
WITH tablejoin AS (
    SELECT c_name, c_custkey, o_custkey, o_totalprice FROM customer
    JOIN orders ON customer.c_custkey = orders.o_custkey
    WHERE o_totalprice > 100000
)

SELECT DISTINCT c_name FROM tablejoin
ORDER BY c_name ASC
LIMIT 10;
"""
cursor.execute(QUERY_CHALLENGE_7)
rows = cursor.fetchall()
print(as_dataframe(rows, cursor).to_string(index=False))

            C_NAME
Customer#000000001
Customer#000000002
Customer#000000004
Customer#000000005
Customer#000000007
Customer#000000008
Customer#000000010
Customer#000000011
Customer#000000013
Customer#000000014


# Submit

🚨 **Make sure you save your notebook (with all outputs)** before submitting! This will ensure the most up-to-date version is graded.

In [11]:
from submit import send_notebook
response = send_notebook("./sql_practice.ipynb")
print(response["response"])


----------------------------------------
Username: david.buzzell@kellogg.northwestern.edu
Uploaded: 2025-04-18T04:07:57Z
Due: 2025-04-18T04:59:59Z
Homework: sql_practice.ipynb
Score: 100%
Best Score: 100%
Attempts Remaining: 1
Late Penalty: 0
----------------------------------------
sql_practice_test.py::TestQueryChallenges::test_query_challenge_1 PASSED [ 14%]
sql_practice_test.py::TestQueryChallenges::test_query_challenge_2 PASSED [ 28%]
sql_practice_test.py::TestQueryChallenges::test_query_challenge_3 PASSED [ 42%]
sql_practice_test.py::TestQueryChallenges::test_query_challenge_4 PASSED [ 57%]
sql_practice_test.py::TestQueryChallenges::test_query_challenge_5 PASSED [ 71%]
sql_practice_test.py::TestQueryChallenges::test_query_challenge_6 PASSED [ 85%]
sql_practice_test.py::TestQueryChallenges::test_query_challenge_7 PASSED [100%]
----------------------------------------

